# Jira Comments – LLM-Assisted Thematic Analysis

1. **Load and clean** Jira comments exported from your PowerShell script.  
2. **Normalize** multiple comments per SWIFT (issue key) while keeping the link between each comment and its issue.  
3. **De-identify** and prepare the comment text for upload to an LLM (e.g. Copilot / GPT) for theme discovery and labeling.  
4. (Optional) **Join back** LLM-generated labels with the comments for further analysis and visualization.


## 1. Setup & Imports

In [ ]:
# Cell 1 – Imports and basic config

import os
import re
import json
import pandas as pd
import numpy as np

pd.set_option("display.max_colwidth", 200)

# OpenAI import for automated theme analysis
from openai import OpenAI

def analyze_themes_automated(comments_df, num_themes=10):
    """Automated theme discovery using LLM API"""
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    # Prepare prompt
    prompt = f"""Analyze these Jira comments about project delays.
    
{comments_df['text_clean'].to_string()}

Identify {num_themes} high-level themes. Return JSON:
{{
  "themes": [
    {{
      "theme": "Theme name",
      "description": "One sentence description",
      "examples": ["phrase 1", "phrase 2", "phrase 3"]
    }}
  ]
}}"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)

## 2. Load the Jira comments CSV

Update the `csv_path` below to point to your exported `JiraComments_FromJql.csv` (or similar).

In [ ]:
# Cell 2 – Load CSV

csv_path = r"C:\Users\Andy\Desktop\Text_Analysis\JiraComments_FromJql.csv"  # <-- change this to your actual path

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV not found at: {csv_path}\nPlease update csv_path to the correct location.")

df = pd.read_csv(csv_path)

print("Rows:", len(df))
print("Columns:", list(df.columns))
df.head(30)

## 3. Normalize structure: one row = one comment

Here we:

- Identify the likely **issue key column** (e.g. `IssueKey`).  
- Identify the **comment body column** (e.g. `Body`).  
- Keep those plus any IDs that help us track comments later (e.g. `CommentId`).  
- Ensure that each row is a single comment with its associated SWIFT / Jira issue key.


In [ ]:
# Cell 3 – Normalize and select key columns

possible_issue_cols = ["IssueKey", "issueKey", "Key", "key"]
possible_body_cols  = ["Body", "body", "Comment", "comment", "Text", "text"]
possible_id_cols    = ["CommentId", "commentId", "Id", "id"]
possible_time_cols  = ["Created", "created", "Timestamp", "timestamp", "Updated", "updated"]

issue_col = next((c for c in possible_issue_cols if c in df.columns), None)
body_col  = next((c for c in possible_body_cols  if c in df.columns), None)
id_col    = next((c for c in possible_id_cols    if c in df.columns), None)
time_col  = next((c for c in possible_time_cols  if c in df.columns), None)

if issue_col is None or body_col is None:
    raise ValueError(
        f"Could not find issue or body column.\n"
        f"Available columns: {list(df.columns)}\n"
        f"Expected something like {possible_issue_cols} and {possible_body_cols}"
    )

print(f"Using issue column : {issue_col}")
print(f"Using body column  : {body_col}")
print(f"Using ID column    : {id_col if id_col else '(none – will create one)'}")
print(f"Using time column  : {time_col if time_col else '(none found)'}")

# Build a normalized DataFrame
cols_to_keep = [issue_col, body_col]
if id_col:
    cols_to_keep.append(id_col)
if time_col:
    cols_to_keep.append(time_col)

data = df[cols_to_keep].copy()
data.rename(columns={issue_col: "IssueKey", body_col: "Body"}, inplace=True)
if time_col:
    data.rename(columns={time_col: "Timestamp"}, inplace=True)

# Create a synthetic comment_id if none exists
if id_col is None:
    data.insert(0, "CommentSeq", np.arange(1, len(data) + 1))
else:
    data.rename(columns={id_col: "CommentId"}, inplace=True)
    if "CommentId" not in data.columns:
        data.insert(0, "CommentSeq", np.arange(1, len(data) + 1))

# Ensure Body is string and non-empty
data["Body"] = data["Body"].astype(str).str.strip()
data = data[data["Body"].str.len() > 0].reset_index(drop=True)

# If we have a timestamp, parse and sort by IssueKey + Timestamp
if "Timestamp" in data.columns:
    data["Timestamp"] = pd.to_datetime(data["Timestamp"], errors="coerce")
    data = data.sort_values(["IssueKey", "Timestamp"], ascending=[True, True])
    data = data.reset_index(drop=True)

print("\nAfter normalization:")
print("Rows (comments):", len(data))
print("Distinct SWIFT / issue keys:", data["IssueKey"].nunique())

data.head(10)

## 4. De-identification & text cleaning

We will:

- Remove Jira user mentions like `[~USER]`.  
- Remove obvious CRQ numbers (`CRQ1234567`).  
- Remove email addresses.  
- Optionally remove issue keys themselves from the comment text.  
- Keep a clean text column suitable for sending to an LLM.

We **do not** touch the `IssueKey` column itself so we can always trace a comment back to its SWIFT later.


In [ ]:
# Cell 4 – De-identification and cleaning

mention_pattern    = re.compile(r"\[~[^\]]+\]")         # [~username]
crq_pattern        = re.compile(r"CRQ\d+", re.IGNORECASE)
email_pattern      = re.compile(r"[\w\.\-]+@[\w\.-]+")
whitespace_pattern = re.compile(r"\s+")

def clean_and_deidentify(row):
    text = str(row["Body"])

    # Remove Jira mentions like [~SMULLIN]
    text = mention_pattern.sub(" ", text)

    # Remove CRQ-style change request numbers
    text = crq_pattern.sub(" ", text)

    # Remove email addresses
    text = email_pattern.sub(" ", text)

    # Remove the explicit issue key text if it appears inside the body
    issue_key = str(row["IssueKey"])
    if issue_key and issue_key in text:
        text = text.replace(issue_key, " ")

    # Remove extra non-informative characters
    # (Keep basic punctuation; strip control chars)
    text = re.sub(r"[^a-zA-Z0-9\s,.!?'-]", " ", text)

    # Normalize whitespace and lowercase
    text = whitespace_pattern.sub(" ", text).strip().lower()

    return text

data["text_clean"] = data.apply(clean_and_deidentify, axis=1)

data[["IssueKey", "Body", "text_clean"]].head(10)

### Quick sanity check

Let’s look at a few random comments to ensure:

- They are de-identified.  
- They still make sense semantically (so an LLM can interpret them).

In [ ]:
# Cell 5 – Random sample for manual inspection

data.sample(min(10, len(data)), random_state=42)[["IssueKey", "text_clean"]]

In [ ]:
# Set your OpenAI API key (REPLACE WITH YOUR ACTUAL KEY!)
os.environ["OPENAI_API_KEY"] = "sk-proj-YOUR-NEW-KEY-HERE"

# Run automated theme discovery
print("🤖 Starting automated theme analysis...")
print(f"📊 Analyzing {len(data)} comments from {data['IssueKey'].nunique()} issues...\n")

themes_result = analyze_themes_automated(data, num_themes=10)

# Display results
print("\n" + "="*80)
print("DISCOVERED THEMES")
print("="*80)
for i, theme in enumerate(themes_result["themes"], 1):
    print(f"\n{i}. {theme['theme']}")
    print(f"   {theme['description']}")
    print(f"   Examples: {', '.join(theme['examples'][:2])}")
    
print(f"\n✅ Theme discovery complete!")

## 7. Automated Theme Discovery with OpenAI

Now we'll use the OpenAI API to automatically discover themes instead of manual copying.

## 5. Prepare text for LLM theme discovery

Now we’ll format the comments into a prompt-friendly structure you can paste into Copilot / GPT.

For example, we’ll generate something like:

```text
1. [Issue SWIFT-7544] this was not ready for prd by 10/27 ...
2. [Issue SWIFT-7543] adding the november release ...
...
```

You can then paste that list into your LLM with the prompt:

> “Here are Jira comments about why work wasn’t completed in October.  
> Read all of them and propose 5–10 high-level themes, each with:  
> • A short title  
> • A one-sentence description of the reason for delay  
> • A few example phrases that fit that theme.  
> Return JSON with keys: `theme`, `description`, `examples`.”


In [ ]:
# Cell 6 – Build grouped, ordered comments for LLM input

rows_for_llm = []

# We'll group by IssueKey, and within each group the rows are already sorted by Timestamp
for issue, grp in data.groupby("IssueKey", sort=True):
    rows_for_llm.append(f"Issue {issue}:")
    for _, row in grp.iterrows():
        ts = row["Timestamp"] if "Timestamp" in row and pd.notna(row["Timestamp"]) else None
        txt = row["text_clean"]

        if ts is not None:
            ts_str = ts.strftime("%Y-%m-%d %H:%M")
            rows_for_llm.append(f"  - [{ts_str}] {txt}")
        else:
            rows_for_llm.append(f"  - {txt}")
    rows_for_llm.append("")  # blank line between issues

llm_input_block = "\n".join(rows_for_llm)

print(llm_input_block)  # show the first ~4000 characters


# Prompt

1. Copy the printed `llm_input_block` text above (the numbered list of comments).  
2. Paste it into your LLM with a prompt like:

> “Here are Jira comments about why work wasn’t completed in October.  
> Read all of them and propose 5–10 high-level themes, each with:  
> • A short title  
> • A one-sentence description of the reason for delay  
> • A few example phrases that fit that theme.  
> Return JSON with keys: `theme`, `description`, `examples`.”

3. Review and possibly edit the returned themes and descriptions.  
4. Decide on your final theme list (e.g. in a separate document or directly in the next cell).


## 6. (Optional) Assign themes back to each comment

After the LLM proposes themes and you finalize them, you can also ask the LLM to **assign a theme to each comment**.

### Example prompt for assignment

For each comment (or for small batches), you might say:

> “Here is a Jira comment about why work wasn’t completed:  
> `<comment text>`  
>  
> Based on the following themes:  
> 1. CRQ timing / missed release window  
> 2. Waiting on other team / vendor  
> 3. Approvals / leadership not finalized  
> 4. Requirements unclear / design not complete  
> 5. Testing / QA / UAT delays  
> 6. Capacity / competing priorities  
> 7. Other / unclear  
>  
> Return only the number and name of the single best-fitting theme.”

You can copy the numbered list of comments and ask the LLM to return a simple table or JSON mapping `comment_number -> theme`.


In [ ]:
import pandas as pd, json

with open("C:/Users/Andy/Desktop/Text_Analysis/JSON_themes.json", "r") as f:
    json_data = json.load(f)  # Changed from 'data' to 'json_data'

themes = pd.DataFrame(json_data["themes"])
issues = pd.DataFrame(json_data["issues"])

summary = {
    "Total Issues": len(issues),
    "Total Themes": len(themes),
    "Avg Themes per Issue": issues["themes"].apply(len).mean()
}
pd.DataFrame([summary])

In [ ]:
import json
import os
from collections import Counter
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

json_path = r"C:\Users\Andy\Desktop\Text_Analysis\JSON_themes.json"

with open(json_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)  # Changed from 'data' to 'json_data'

# Adjust these keys if your JSON is shaped differently
themes_raw = json_data.get("themes", [])
issues_raw = json_data.get("issues", [])

themes_df = pd.DataFrame(themes_raw)
issues_df = pd.DataFrame(issues_raw)

print("Themes:", themes_df.shape)
print("Issues:", issues_df.shape)
themes_df.head(), issues_df.head()

In [ ]:
# Count how many issues each theme appears in
all_theme_mentions = [t for theme_list in issues_df["themes"] for t in theme_list]
theme_counts = Counter(all_theme_mentions)

theme_freq = (
    pd.DataFrame(theme_counts.items(), columns=["Theme", "Count"])
    .sort_values("Count", ascending=False)
)

theme_freq


In [ ]:
# Bar chart
plt.figure(figsize=(10, 6))

plt.barh(theme_freq["Theme"], theme_freq["Count"])
plt.gca().invert_yaxis()  # largest at top

plt.title("Top Reasons for October Jira Delays (by # of Issues)")
plt.xlabel("Number of Issues")
plt.ylabel("Theme")

plt.tight_layout()
plt.show()
